In [2]:
import requests
import codecs
import csv
from bs4 import BeautifulSoup 
import json
import io
import gzip
from html.parser import HTMLParser

In [3]:
get_requests1 = requests.get('http://index.commoncrawl.org/CC-MAIN-2019-09-index?url=http%3A%2F%2Fhttps://www.sportingnews.com/&output=json')
get_requests2 = requests.get('http://index.commoncrawl.org/CC-MAIN-2019-04-index?url=http%3A%2F%2Fhttps://www.sportingnews.com/&output=json')
requestsArray = []
requestsArray.append(get_requests1)
requestsArray.append(get_requests2)

In [4]:
pageList = []
for r in requestsArray:
    if r.status_code == 200:
        pages = r.content.splitlines()
        for i in pages:
            pageList.append(json.loads(i))

print(pages)
#pages = [json.loads(x) for x in get_requests.content.decode('utf8').strip().split('\n')]
#page = pages[2]

[b'{"urlkey": "com,sportingnews)/", "timestamp": "20190116061044", "digest": "AXEC5IAN3RFEASW6L6KYD5NPCNDJ2W3V", "offset": "361289283", "length": "38201", "url": "http://www.sportingnews.com/", "languages": "eng", "filename": "crawl-data/CC-MAIN-2019-04/segments/1547583656897.10/warc/CC-MAIN-20190116052151-20190116074151-00371.warc.gz", "mime": "text/html", "charset": "UTF-8", "status": "200", "mime-detected": "text/html"}', b'{"urlkey": "com,sportingnews)/", "timestamp": "20190116164204", "digest": "BHS2YP36XJKPP54PL7GMKNZZCMWL4DRW", "offset": "380192857", "length": "38352", "url": "http://www.sportingnews.com/", "languages": "eng", "filename": "crawl-data/CC-MAIN-2019-04/segments/1547583657555.87/warc/CC-MAIN-20190116154927-20190116180927-00371.warc.gz", "mime": "text/html", "charset": "UTF-8", "status": "200", "mime-detected": "text/html"}', b'{"urlkey": "com,sportingnews)/", "timestamp": "20190116192205", "digest": "XLZW737NFBEOSCLLNMQZOI4SSBS2BASL", "offset": "382260440", "length"

In [5]:
page = pageList[1]
print(page)

{'urlkey': 'com,sportingnews)/', 'timestamp': '20190216082010', 'digest': 'Z36AFMJETR45BJGGL4XUTK2NJHQ6CK2X', 'offset': '442613246', 'length': '38768', 'url': 'http://www.sportingnews.com/', 'languages': 'eng', 'filename': 'crawl-data/CC-MAIN-2019-09/segments/1550247479967.16/warc/CC-MAIN-20190216065107-20190216091107-00371.warc.gz', 'mime': 'text/html', 'charset': 'UTF-8', 'status': '200', 'mime-detected': 'text/html'}


In [6]:
offset, length = int(page['offset']), int(page['length'])
offset_end = offset + length - 1
print(offset, length, offset_end)

#prefix = 'https://aws-publicdatasets.s3.amazonaws.com/'
prefix = 'https://commoncrawl.s3.amazonaws.com/'

response = requests.get(prefix + page['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
#response = requests.get("https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2019-09/segments/1550247479627.17/wet/CC-MAIN-20190215224408-20190216010408-00271.warc.wet.gz", headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
print(response)

442613246 38768 442652013
<Response [206]>


In [7]:
raw_data = io.BytesIO(response.content)
print(raw_data)
f = gzip.GzipFile(fileobj=raw_data)
print(f)

<gzip _io.BytesIO object at 0x1152c20f8 0x1129971d0>


In [8]:
data = f.read()
#print(data)
#resp = ""
#if len(data):
#    try:
#        resp = data.decode('utf8').strip().split('\r\n\r\n', 2)
#    except:
#        pass
#print(resp)

In [10]:
#doc = parse(io.BytesIO(data)).getroot()
doc = BeautifulSoup(io.BytesIO(data), 'lxml')
#print(doc)
all_a = doc.find_all('a')
hrefList = []
if all_a:
    for l in all_a:
        #print(l)
        href = l.attrs.get('href')
        
        if href is not None:
            if href.startswith('http'):
                hrefList.append(href)

In [19]:
with codecs.open('links.csv', 'wb', encoding = 'utf8') as output:
    f = ['url']
    write_csv = csv.DictWriter(output, fieldnames = f)
    write_csv.writeheader()
    
    for h in hrefList:
        write_csv.writerow({'url':h})
        
hrefList = list( dict.fromkeys(hrefList) )

for h in hrefList:
    print(h)
        

http://www.sportingnews.com/us
http://www.sportingnews.com/us/nfl/scoreboard
http://www.sportingnews.com/us/nfl
http://www.sportingnews.com/us/nba
http://www.sportingnews.com/us/mlb
http://www.sportingnews.com/us/nhl
http://www.sportingnews.com/us/boxing
http://www.sportingnews.com/us/ncaa-football
http://www.sportingnews.com/us/ncaa-basketball
http://www.sportingnews.com/us/mma
http://bit.ly/2NYRJKC
http://www.sportingnews.com/us/fantasy
http://www.sportingnews.com/us/nascar
http://www.sportingnews.com/us/soccer
http://www.sportingnews.com/us/tennis
http://www.sportingnews.com/us/athletics
http://www.sportingnews.com/us/golf
http://www.sportingnews.com/us/wwe
http://www.sportingnews.com/us/other-sports
https://www.facebook.com/sportingnews
https://www.twitter.com/sportingnews
https://instagram.com/sportingnews/
http://www.goal.com
http://www.sportingnews.com/us/writers
http://www.sportingnews.com/us/newsletter/signup
http://shop.sportingnews.com
http://www.sportingnews.com/us/privacy


In [23]:
#hrefList[1]
all_text = ""
for h in hrefList:
     print("---------------")
     print(h)
     response = requests.get(h)
     if response.status_code == 200:
         doc = BeautifulSoup(response.text, 'html.parser')
         name_box = doc.find_all('p')

         for name in name_box:
            for x in name.find_all('a'):
                x.replaceWithChildren()

         for link in name_box:
            all_text+=link.text
     #print("----------------------------------------------------")
     #print(all_text)

print(all_text)
    
#p = requests.get(hrefList[1])
#print(p.status_code)
#print(p.text)
'''pageListFinal = []
if p.status_code == 200:
    pagesFinal = p.content
    print(pagesFinal)
    for i in pagesFinal:
        pageListFinal.append(json.loads(i))'''
        
#print(pageListFinal[0])

---------------
http://www.sportingnews.com/us
---------------
http://www.sportingnews.com/us/nfl/scoreboard
---------------
http://www.sportingnews.com/us/nfl
---------------
http://www.sportingnews.com/us/nba
---------------
http://www.sportingnews.com/us/mlb
---------------
http://www.sportingnews.com/us/nhl
---------------
http://www.sportingnews.com/us/boxing
---------------
http://www.sportingnews.com/us/ncaa-football
---------------
http://www.sportingnews.com/us/ncaa-basketball
---------------
http://www.sportingnews.com/us/mma
---------------
http://bit.ly/2NYRJKC
---------------
http://www.sportingnews.com/us/fantasy
---------------
http://www.sportingnews.com/us/nascar
---------------
http://www.sportingnews.com/us/soccer
---------------
http://www.sportingnews.com/us/tennis
---------------
http://www.sportingnews.com/us/athletics
---------------
http://www.sportingnews.com/us/golf
---------------
http://www.sportingnews.com/us/wwe
---------------
http://www.sportingnews.com

'pageListFinal = []\nif p.status_code == 200:\n    pagesFinal = p.content\n    print(pagesFinal)\n    for i in pagesFinal:\n        pageListFinal.append(json.loads(i))'

In [21]:
print(response.status_code)

200


In [15]:
#print(all_text)
text_file = open("Common_crawl_Sportingnews_com_text_11_Apr.txt", "w")
text_file.write( all_text)
text_file.close()

In [16]:
 response = requests.get("http://shop.sportingnews.com/COLLEGE")
 doc = BeautifulSoup(response.text, 'html.parser')
 #print(doc)
    
 name_box = doc.find_all('p')
 all_text = ""
 for name in name_box:
    #print(".................")
    #print(name)
    
    for x in name.find_all('a'):
        x.replaceWithChildren()
        #all_text += (x.get_text()) 
        #print(all_text)
    #print("%%%%%%%%%%%")
    #print(name)
    #all_text += name
    #print(all_text)

for link in name_box:
    all_text+=link.text
print(all_text)

The Villanova Wildcats reigned supreme in March Madness, so gear up for the champs with our wide range of NCAA Basketball Apparel including official Villanova National Champs Gear that allow Wildcats fans to ring in their second title in three years! Score more than just College Basketball Gear at Fanatics, as we're always to stock your closet with NCAA Football Apparel and College Baseball Gear. Fanatics.com is your ultimate College Apparel store, bringing Students, Fans and Alumni all the College Gear they could ever need. With a huge selection of College Jerseys, College Clothing, NCAA Sweatshirts and T-Shirts, every die-hard fan or alumnus can find a way to show their school pride. We offer College Sportswear for over 500 schools, so you can be sure you're checking out the largest collection of officially licensed College Gear and NCAA Merchandise anywhere. Our NCAA Shop houses an endless variety of College Jerseys so you'll be totally set for every season. 

                  Shop